# PageRank

In [ ]:
# Instalamos PySpark
!pip install pyspark

### IMPORTAMOS DESDE DRIVE (CAMBIAR LA BASE DE DATOS SI ES NECESARIA [PORFAVOR RESPETAR LA ESTRUCTURA O ME PONDRE TRISTE])

In [ ]:
# En este caso, nosotros tenemos cora en nuestro google drive, ustedes deben poner su ruta para cora.cites
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
# En este caso, nosotros tenemos cora en nuestro google drive, ustedes deben poner su ruta para cora.cites
df = pd.read_csv('/content/drive/MyDrive/cora.cites',sep="\t",
    header=None,
    names=["target", "source"]) 

In [ ]:
import numpy as np

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
# Pasamos el df anterior a listas para procesarlos en nuestro algoritmo.

# Obtener edges
edges = list(zip(df['source'], df['target']))

# Obtener nodos
nodes = list(set(df['source']).union(set(df['target'])))

print(edges)
print(nodes)

In [ ]:
rdd_nodes = spark.sparkContext.parallelize(nodes)
rdd_edges = spark.sparkContext.parallelize(edges)

In [ ]:
# Esta funcion se encarga de asignar el PageRank inicial para cada nodo
def asignar_pagerank(nodo, cantidad_nodos):
    return (nodo, 1 / cantidad_nodos)


cantidad_nodos = rdd_nodes.count()
rdd_nodes = rdd_nodes.map(lambda nodo: asignar_pagerank(nodo, cantidad_nodos))

In [ ]:
# Conteo se encarga de contar las aristas asociadas a un nodo (Solo aristas que salen desde el nodo, no las que llegan al nodo)
conteo = rdd_edges.map(lambda x: (x[0], 1)).reduceByKey(lambda a, b: a + b)

In [ ]:
# Los mensajes corresponden a tuplas las cuales tienen la siguiente forma (nodo, mensajes_recibidos)

def mensajes_inicialesrdd(rdd_nodes,conteo):
  rdd_salidas = rdd_nodes.map(lambda x: (x[0], x[1])).join(conteo)
  rdd_salidas = rdd_salidas.map(lambda tupla: (tupla[0],tupla[1][0]/tupla[1][1]))
  return rdd_salidas

rdd_mensajes = mensajes_inicialesrdd(rdd_nodes,conteo)

In [ ]:
# Hacemos un Join entre los mensajes y las aristas
join_rdds = rdd_mensajes.join(rdd_edges).map(lambda line: (line[1][1], line[1][0])) \
            .reduceByKey(lambda a, b: a + b)

In [ ]:
# Rellenamos aquellos nodos que no se incluyeron debido a que no reciben mensajes
nodes_with_zero = rdd_nodes.map(lambda x: (x[0], 0))

In [ ]:
# Creamos un RDD final para trabajar en el algoritmo
rdd_final = join_rdds.union(nodes_with_zero).reduceByKey(lambda a, b: a + b).map(lambda x: (x[0], 1/4, x[1]))

In [ ]:
# Nuestro dumping factor va a ser considerado como 0.85

# Define el factor de dumping
dumping_factor = 0.85

# Calcula el número total de nodos en el RDD
num_nodos = rdd_final.count()
val_inicial = 1 / num_nodos
# Calcula el factor de redistribución
redistribution_factor = (1 - dumping_factor) / num_nodos

# Función para actualizar los valores de PageRank para cada nodo
def actualizar_page_rank(nodo):
    nodo_id, page_rank, ref = nodo
    page_rank = (1 - dumping_factor)/num_nodos
    new_page_rank =  page_rank + (dumping_factor * ref)
    return (nodo_id, new_page_rank, (dumping_factor * ref))

In [ ]:
# Algorithm es un algoritmo que utiliza actulizar_page_rank
def algorithm(rdd_final):

  dumping_factor = 0.85
  num_nodos = rdd_final.count()
  redistribution_factor = (1 - dumping_factor) / num_nodos
  rdd_actualizado = rdd_final.map(actualizar_page_rank)

  return rdd_actualizado

In [ ]:
# Definimos una iteracion maxima para terminar, en este caso es 10, o su cambio es menor a epsilon.
# El print lo hace del tipo (nodo, PageRank, Redistribution Factor)

count = 0
epsilon = 0.0000001
iteracion_anterior = None
while True:
    rdd_final = algorithm(rdd_final)
    if iteracion_anterior is not None:
        # veamos si los valores de la iteracion anterior y la actual son muy parecidos
        if iteracion_anterior.join(rdd_final).filter(lambda x: x[1][0] !=x [1][1]).filter(lambda x: abs(x[1][0] - x[1][1]) < epsilon).count() != 0:
            break
    print(rdd_final.collect())
    count += 1
    iteracion_anterior = rdd_final
    if count == 10:
        break